In [2]:
!pip install pendulum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 21.1 MB/s eta 0:00:00


In [22]:
#Import Library
import numpy as np
import pandas as pd
from datetime import datetime
import calendar 
import pendulum
from openpyxl import load_workbook

#Import Data
data_raw = pd.read_excel('Incoming 10423.xlsx', sheet_name='Sheet1')
data_sku_product = pd.read_excel('SKU.xlsx', sheet_name='Product')
data_sku_sales = pd.read_excel('SKU.xlsx', sheet_name='Sales')
data_sku_inventory = pd.read_excel('SKU.xlsx', sheet_name='Inventory')
data_sku_mapping = pd.read_excel('SKU.xlsx', sheet_name='Mapping')
data_buying_plan = pd.read_excel('Buying Plan.xlsx', sheet_name='Sheet1')

#Create Temporary List
gen_norm = []
brand_code = []
stock_qty = []
month2 = []
year = []
division = []
subdiv_code = []
subdiv = []
gender = []
week = []
brand_name = []
vendor_art = []
cost = []
stock_cost = []
month_desc = []
remarks = []
rrp = []
stock_retail = []

#Missing Variables
missing_store_site = []
missing_article_product = []

#Delete Rows When Article is Blank
data_raw = data_raw.dropna(subset=['Article'])
data_raw = data_raw.reset_index(drop=True)

#Copy Data Raw
data_raw2 = data_raw.copy(deep=True)

#Filter Movement Type
data_raw = data_raw.drop(data_raw[data_raw['Movement Type'].isin([641,642, 351])].index)  
data_raw = data_raw.reset_index(drop=True)

data_raw2 = data_raw2.drop(data_raw[data_raw['Movement Type'].isin([101,102, 603, 604])].index)  
data_raw2 = data_raw2.reset_index(drop=True)

for i in range(len(data_raw)) : 
  #Month and Month Desc
  month_form = str(data_raw['Posting Date'][i].month).zfill(2) + " - " + calendar.month_abbr[int(data_raw['Posting Date'][i].month)]
  month2.append(int(data_raw['Posting Date'][i].month))
  month_desc.append(month_form)

  #Year
  year.append(int(data_raw['Posting Date'][i].year))

  #Week
  num_week = int(data_raw['Posting Date'][i].strftime("%U"))
  if 49 <= num_week >= 52 : 
    num_week += 48
  else : 
    num_week -= 4
  num_week = str(num_week).zfill(2)
  week_name = "Wk " + num_week
  week.append(week_name)

  #Normalize GenArt, Brand Code, and Stock Norm
  gen_art = str(data_raw['Article'][i])
  brand = str(data_raw['Article'][i])[0:3]
  brand_code.append(brand)
  
  if gen_art.find('MB-') == -1 : 
    check_art = 0
    for j in range(8,13) : 
      art = gen_art[0:j]
      art_loc = data_sku_product.index[data_sku_product['Gen.art']==art].tolist()
      if len(art_loc) > 0 and check_art == 0:
        check_art = 1
        gen_norm.append(art)
        stock_qty.append(int(data_raw['Quantity'][i]))
      else : 
        art_loc = data_buying_plan.index[data_buying_plan['SAP Article']==art].tolist()
        if len(art_loc) > 0 and check_art == 0:
          check_art = 1
          gen_norm.append(art)
          stock_qty.append(int(data_raw['Quantity'][i]))
      if j==12 and check_art == 0: 
        gen_norm.append("#N/A")
        stock_qty.append("#N/A")
  else : 
    gen_art = gen_art[0:3] + gen_art[6:(6+(len(gen_art)-9))]
    gen_norm.append(gen_art)
    if brand == 'JDK' or brand == 'NYA' or brand == 'CVK' :
      lookup_value = str(data_raw['Article'][i])[-3:]
      stock_loc = data_sku_mapping.index[data_sku_mapping['Code 2']==lookup_value].tolist()
      if len(stock_loc)>0 :
        stock_qty.append(int(data_sku_mapping['qty'][stock_loc[0]])*int(data_raw['Quantity'][i]))
      else :
        missing_article_product.append(gen_art)
        stock_qty.append("#N/A")
    else :
      lookup_value = brand + str(data_raw['Article'][i])[-3:]
      stock_loc = data_sku_mapping.index[data_sku_mapping['Code 1']==lookup_value].tolist()
      if len(stock_loc)>0 :
        stock_qty.append(int(data_sku_mapping['qty'][stock_loc[0]])*int(data_raw['Quantity'][i]))
      else :
        missing_article_product.append(gen_art)
        stock_qty.append("#N/A")
  
  #Division to Color
  #FL Indonesia
  genarticle_loc = data_sku_product.index[data_sku_product['Gen.art']==gen_norm[i]].tolist()

  #FL Philippines
  # genarticle_loc = data_sku_product.index[data_sku_product['Gen.art (PH)']==gen_norm[i]].tolist()

  if len(genarticle_loc)>0 :
    division.append(data_sku_product['FL Division'][genarticle_loc[0]])
    subdiv_code.append(data_sku_product['FL Subdivision Code'][genarticle_loc[0]])
    subdiv.append(data_sku_product['FL Subdivision'][genarticle_loc[0]])
    gender.append(data_sku_product['FL Gender / Age'][genarticle_loc[0]])
    vendor_art.append(data_sku_product['Vendor Art'][genarticle_loc[0]])
    brand_name.append(data_sku_product['Brand Name'][genarticle_loc[0]])
  else :
    missing_article_product.append(gen_norm[i])
    division.append("#N/A")
    subdiv_code.append("#N/A")
    subdiv.append("#N/A")
    gender.append("#N/A")
    vendor_art.append("#N/A")
    brand_name.append("#N/A")
  
  #Cost and Stock Cost
  if int(float(data_raw['Amount in LC'][i])) == 0 :
    lookup_value = gen_norm[i]
    cost_loc = data_raw2.index[data_raw2['Amount in LC']==lookup_value].tolist()
    check_cost = 0 
    for j in range(len(cost_loc)) : 
      if data_raw2['Amount in LC'][cost_loc[j]] != 0 and check_cost == 0: 
        check_cost = 1
        cost.append(data_raw2['Amount in LC'][cost_loc[j]])
        stock_cost.append(int(float(data_raw2['Amount in LC'][cost_loc[j]])) * int(float(data_raw['Quantity'][i])))
      else: 
        cost_loc = data_buying_plan.index[data_buying_plan['SAP Article']==lookup_value].tolist()
        if len(cost_loc)>0 and check_cost==0:
          check_cost = 1
          cost.append(int(data_buying_plan['Est Landed Cost'][cost_loc[0]]))
          stock_cost.append(int(float(data_buying_plan['Est Landed Cost'][cost_loc[0]])) * int(float(data_raw['Quantity'][i])))
    if check_cost==0 :
      missing_article_product.append(gen_art)
      cost.append("#N/A")
      stock_cost.append("#N/A")
  else :
    cost.append(int(float(data_raw['Amount in LC'][i]))/int(float(data_raw['Quantity'][i])))
    stock_cost.append(int(float(data_raw['Amount in LC'][i])))

  #Stock Retail
  if int(float(data_raw['Sales Value inc. VAT'][i])) == 0:
    lookup_value = gen_norm[i]
    retail_loc = data_raw2.index[data_raw2['Sales Value inc. VAT']==lookup_value].tolist()
    check_retail = 0 
    for j in range(len(retail_loc)) : 
      if data_raw2['Sales Value inc. VAT'][retail_loc[j]] != 0 and check_retail == 0: 
        check_retail = 1
        stock_retail.append(data_raw2['Sales Value inc. VAT'][retail_loc[j]])
    if check_retail==0 : 
      stock_retail.append("#N/A") #belum cek rrp
  else :
    stock_retail.append(int(float(data_raw['Sales Value inc. VAT'][i])))

  #Remarks and RRP
  remarks.append(" ")
  rrp.append(" ")

#Combine All
data_final = data_raw.copy(deep=True)
data_final['Gen Norm'] = gen_norm
data_final['Brand'] = brand_code
data_final['Qty Norm'] = stock_qty
data_final['Month'] = month2
data_final['Year'] = year
data_final['FL Division'] = division
data_final['FL SubDiv Code'] = subdiv_code
data_final['FL SubDiv'] = subdiv
data_final['FL Gender / Age'] = gender
data_final['Week FL'] = week
data_final['Brand Name'] = brand_name
data_final['Vendor Art'] = vendor_art
data_final['Cost'] = cost
data_final['Stock Cost'] = stock_cost
data_final['Month Desc'] = month_desc
data_final['BRAND'] = brand_name
data_final['Remarks'] = remarks
data_final['RRP'] = rrp
data_final['Stock Retail'] = stock_retail

#Export Data
data_final.to_excel("Hasil Incoming.xlsx", sheet_name = 'FINAL')

if len(missing_store_site) > 0:
  print("Terdapat Missing Store Site : %d Baris" % (len(missing_store_site)))
  missing_store_site = pd.DataFrame(missing_store_site, columns =['Store Site'])
  missing_store_site.to_excel('Missing Store Site.xlsx', sheet_name='Store Site Missing')
else : 
  print("Tidak Terdapat Missing Store Site : %d Baris" % (len(missing_store_site)))

if len(missing_article_product) > 0:
  print("Terdapat Missing Article Product : %d Baris" % (len(missing_article_product)))
  missing_article_product = pd.DataFrame(missing_article_product, columns =['Article Product'])
  missing_article_product.to_excel('Missing Article Product.xlsx', sheet_name='Article Product Missing')
else : 
  print("Tidak Missing Article Product : %d Baris" % (len(missing_article_product)))

Tidak Terdapat Missing Store Site : 0 Baris
Terdapat Missing Article Product : 1718 Baris
